In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import os
import IPython.display as ipd
import librosa
import math
import copy

/home/ivantllo/mamba_evaluation/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
len(os.listdir('data/youtubemix/piano'))

1806

In [3]:
fname = 'data/youtubemix/piano/0.wav'   # Raindrop
ipd.Audio(fname)

In [4]:
data, _ = librosa.core.load('data/youtubemix/piano/0.wav', sr=96000, res_type='kaiser_fast')

In [5]:
data[100000:100010]

array([-0.09687651, -0.09795085, -0.09887539, -0.09964494, -0.10025493,
       -0.10069937, -0.10097206, -0.101067  , -0.10097878, -0.10070263],
      dtype=float32)

In [6]:
data.shape

(768000,)

In [7]:
# DataGenerator class to load .wav files with librosa and return a PyTorch tensor loaded in cuda in batches
class DataGenerator(torch.utils.data.Dataset):
    def __init__(self, files, sr=96000):
        self.files = files
        self.sr = sr

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        data, _ = librosa.core.load(self.files[idx], sr=self.sr, res_type='kaiser_fast')
        return torch.tensor(data, dtype=torch.float32).to("cuda")

In [8]:
# DataLoader to load torch tensors in batches
def get_dataloader(files, batch_size=8):
    dataset = DataGenerator(files)
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Load all .wav files in the directory
files = ['data/youtubemix/piano/' + f for f in os.listdir('data/youtubemix/piano') if f.endswith('.wav')]
train_files, test_files = train_test_split(files, test_size=0.2)

train_loader = get_dataloader(train_files)
test_loader = get_dataloader(test_files)

In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [10]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [11]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [12]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [13]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [14]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [20]:
src_vocab_size = 2 ** 8
tgt_vocab_size = 2 ** 8
d_model = 1
num_heads = 1
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout).cuda()

In [21]:
input_pos_embedding = torch.nn.Embedding(1024, embedding_dim=d_model)
target_pos_embedding = torch.nn.Embedding(1024, embedding_dim=d_model)

encoder_layer = nn.TransformerEncoderLayer(
    d_model=d_model,
    nhead=num_heads,
    dropout=dropout,
    dim_feedforward=4 * d_model,
)
decoder_layer = nn.TransformerDecoderLayer(
    d_model=d_model,
    nhead=num_heads,
    dropout=dropout,
    dim_feedforward=4 * d_model,
)

encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
decoder = torch.nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

input_projection = nn.Linear(src_vocab_size, d_model)
output_projection = nn.Linear(src_vocab_size, d_model)

linear = nn.Linear(d_model, 1)
do = nn.Dropout(p=dropout)

In [22]:
# Define the criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(transformer.parameters(), lr=1e-3)

In [23]:
NUM_EPOCHS = 10

In [24]:
# Train the model
for epoch in range(NUM_EPOCHS):
    transformer.train()
    total_loss = 0
    for data in train_loader:
        data = data.reshape(-1, data.shape[1], 1)
        optimizer.zero_grad()
        y = transformer(data[:, :-1], data[:, 1:])
        loss = criterion(y[:, :-1].reshape(-1, y.size(2)), data[:, 1:].reshape(-1))
        # loss = criterion(y, data)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch}: Loss = {total_loss}")

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 2359289856004 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
# Predict the output for the test data
with torch.no_grad():
    transformer.eval()
    y_pred = []
    y_true = []
    for data in test_loader:
        data = data.reshape(-1, data.shape[1], 1)
        y = transformer(data)
        y_pred.append(y.detach().cpu().numpy())
        y_true.append(data.detach().cpu().numpy())
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)

In [ ]:
# Save model
torch.save(transformer.state_dict(), 'models/audio_transformer_model.pth')

In [ ]:
# Load model
transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)
transformer.load_state_dict(torch.load('models/audio_transformer_model.pth'))

<All keys matched successfully>

In [ ]:
# Predict the output for the test data
with torch.no_grad():
    model_loaded.eval()
    y_pred = []
    y_true = []
    for data in test_loader:
        data = data.reshape(-1, data.shape[1], 1)
        y = model_loaded(data)
        y_pred.append(y.detach().cpu().numpy())
        y_true.append(data.detach().cpu().numpy())
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)

In [ ]:
y_pred[0].shape

(768000, 1)

In [ ]:
y_pred[0][350000:350010]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [ ]:
torchaudio.save('data/youtubemix/output.wav', torch.tensor(y_pred[0], dtype=torch.float32), 96000)

In [ ]:
ipd.Audio('data/youtubemix/output.wav')